<a href="https://colab.research.google.com/github/vignz/python/blob/master/PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Removing Java 11 as Pyspark requires Java ***8***

---



In [0]:
!dpkg-query -W -f='${binary:Package}\n' | grep -E -e '^(ia32-)?(sun|oracle)-java' -e '^openjdk-' -e '^icedtea' -e '^(default|gcj)-j(re|dk)' -e '^gcj-(.*)-j(re|dk)' -e '^java-common' | xargs sudo apt-get -y remove
!sudo apt-get -y autoremove
!dpkg -l | grep ^rc | awk '{print($2)}' | xargs sudo apt-get -y purge
!sudo bash -c 'ls -d /home/*/.java' | xargs sudo rm -rf
!sudo rm -rf /usr/lib/jvm/*
!for g in ControlPanel java java_vm javaws jcontrol jexec keytool mozilla-javaplugin.so orbd pack200 policytool rmid rmiregistry servertool tnameserv unpack200 appletviewer apt extcheck HtmlConverter idlj jar jarsigner javac javadoc javah javap jconsole jdb jhat jinfo jmap jps jrunscript jsadebugd jstack jstat jstatd native2ascii rmic schemagen serialver wsgen wsimport xjc xulrunner-1.9-javaplugin.so; do sudo update-alternatives --remove-all $g; done

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-demo-suite-10-0 cuda-documentation-10-0 cuda-drivers
  cuda-nsight-compute-10-0 cuda-runtime-10-0 cuda-samples-10-0 dkms freeglut3
  freeglut3-dev keyboard-configuration libargon2-0 libcap2 libcryptsetup12
  libdevmapper1.02.1 libfontenc1 libidn11 libip4tc0 libjansson4
  libnvidia-common-418 libnvidia-decode-418 libnvidia-decode-430
  libnvidia-encode-418 libnvidia-encode-430 libnvidia-fbc1-418
  libnvidia-fbc1-430 libnvidia-ifr1-418 libnvidia-ifr1-430 libpam-systemd
  libpcsclite1 libpolkit-agent-1-0 libpolkit-backend-1-0 libpolkit-gobject-1-0
  libxfont2 libxi-dev libxkbfile1 libxmu-dev libxmu-headers libxnvctrl0
  libxtst6 nvidia-compute-utils-418 nvidia-compute-utils-430 nvidia-dkms-418
  nvidia-dkms-430 nvidia-driver-418 nvidia-driver-430 nvidia-kernel-common-418
  nvidia-kernel-common-430 nvidia-kern

***Installing Java 8 for Spark***

---



In [0]:
!sudo apt install openjdk-8-jdk
!sudo update-alternatives --config java
!java -version
!pip install pyspark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  ca-certificates-java fonts-dejavu-core fonts-dejavu-extra java-common
  libatk-wrapper-java libatk-wrapper-java-jni libfontenc1 libgail-common
  libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libpcsclite1 libxtst6
  libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre openjdk-8-jre-headless
  x11-utils
Suggested packages:
  default-jre gvfs pcscd openjdk-8-demo openjdk-8-source visualvm
  icedtea-8-plugin libnss-mdns fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  ca-certificates-java fonts-dejavu-core fonts-dejavu-extra java-common
  libatk-wrapper-java libatk-wrapper-java-jni libfontenc1 libgail-common
  libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libpcsclite1 libxtst6
  libxxf86dga1 openjdk-8-jdk openjdk-8-jdk-headless op

***Installing PySpark and Importing libraries***

---




In [0]:
import os
import math
import pyspark
import pyspark.sql.functions as f
from pyspark.sql.types import *
from datetime import datetime
from datetime import timedelta
from pyspark import SparkContext
from pyspark.sql import HiveContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import desc
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql.functions import concat, substring , lit
sc = SparkContext.getOrCreate()
hc = HiveContext(sc)

In [0]:
data = sc.parallelize([("a",1),("a",1),("b",1),("c",1),("d",1),("e",5),("f",5),("g",5)])
data1 = sc.parallelize([("a",(1,2)),("b",(1,3)),("c",1),("d",1),("e",5),("f",5),("g",5)])
schema = StructType([
  StructField("EMP_ID", StringType(), nullable = True),
  StructField("DEP_ID", IntegerType(), nullable = True)
])

df_data = hc.createDataFrame(data, schema=schema)

df_data.show()

+------+------+
|EMP_ID|DEP_ID|
+------+------+
|     a|     1|
|     a|     1|
|     b|     1|
|     c|     1|
|     d|     1|
|     e|     5|
|     f|     5|
|     g|     5|
+------+------+



In [0]:
windowSpec = Window.partitionBy(f.col('DEP_ID')).orderBy(f.col('DEP_ID'))
df_data.withColumn("ROW_NUM", f.rank().over(windowSpec)).show()

+------+------+-------+
|EMP_ID|DEP_ID|ROW_NUM|
+------+------+-------+
|     a|     1|      1|
|     a|     1|      1|
|     b|     1|      1|
|     c|     1|      1|
|     d|     1|      1|
|     e|     5|      1|
|     f|     5|      1|
|     g|     5|      1|
+------+------+-------+



In [0]:
/content/clicklog_2017-03-20.csv

SyntaxError: ignored

In [0]:
schema = StructType([
  StructField("user_id", LongType()),
  StructField("time", StringType()),
  StructField("action", StringType()),
  StructField("destination", StringType()),
  StructField("hotel", StringType())
])

In [0]:
click = hc.read.format('com.databricks.spark.csv').options(header='true', delimiter=',').options(samplingRatio='0.5') \
    .options(encoding='ISO-8859-1').load("file:/content/clicklog_2017-03-20.txt", schema=schema) \
    .persist(pyspark.StorageLevel.MEMORY_AND_DISK)

In [0]:
click.show()

+-------------------+--------+--------+-----------+--------------------+
|            user_id|    time|  action|destination|               hotel|
+-------------------+--------+--------+-----------+--------------------+
|2017032000000000097|00:00:00|   Click|     Rimini|     Regina Elena 57|
|2017032000000000097|00:03:53|   Click|     Rimini|     Regina Elena 57|
|2017032000000000097|00:01:16|   Click|     Rimini|     Regina Elena 57|
|2017032000000000097|00:04:34|   Click|     Rimini|     Regina Elena 57|
|2017032000000000129|00:07:32|   Click|     Berlin|   MÃ¼ggelsee Berlin|
|2017032000000000129|00:18:36|   Click|     Berlin|SensCity Berlin S...|
|2017032000000000129|00:16:12|   Click|     Berlin|Azimut Berlin Cit...|
|2017032000000000129|00:05:04|   Click|    Cottbus|        City Cottbus|
|2017032000000000129|00:01:50|   Click|    Cottbus|              Ostrow|
|2017032000000000129|00:12:05|   Click|     Berlin|Days Inn Berlin West|
|2017032000000000129|00:11:20|   Click|     Berlin|

In [0]:
def sum(a,b):
    return a+b

In [0]:
import unittest
class test_clicklog(unittest.TestCase):
    def test_sum(self):
        a=sum(2,4)
        self.assertEqual(a,6)
        self.assertEqual(a,8)

In [0]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

F
FAIL: test_sum (__main__.test_clicklog)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-27-a59c7894d2ef>", line 6, in test_sum
    self.assertEqual(a,8)
AssertionError: 6 != 8

----------------------------------------------------------------------
Ran 1 test in 0.003s

FAILED (failures=1)


In [0]:
!pip install mock

In [0]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import HiveContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *

from mock import MagicMock
import pyspark.sql.functions as f
#from Dataframe import Dataframe
#from log import Log
import os